In [1]:
import pandas as pd
import glob
import csv
import os

In [2]:
def get_description(maquina):
    df = pd.read_csv(maquina)
    return df.iloc[0]['Descrição']
        
def check_names(nome_lista, maquina_atual):
    return nome_lista in maquina_atual

def filter_name(nome_lista):
    return nome_lista.replace(";", "")

In [3]:
def listagem_maquinas():
    # Lê todos os caminhos para os arquivos das máquinas
    maquinas = glob.glob('maquinas/*.csv')
    
    # Lê a listagem de nomes das máquinas
    nomes = pd.read_csv('nomes/nomes_maquinas.csv')
    
    # Cria arquivo de saida 
    with open('lista.csv', 'w') as csvfile:
        filewriter = csv.writer(
            csvfile, 
            lineterminator='\n',
            delimiter=',',
            quotechar='|', 
            quoting=csv.QUOTE_MINIMAL
        )
        
        # Cria colunas
        filewriter.writerow(['Nome', 'Descricao'])

        # Percorre o nome de todas as máquinas
        for i in range(nomes.shape[0]):
            
            # Limpa o nome da máquina
            nome_puro = filter_name(nomes.iloc[i][0])

            # Procura a máquina nos arquivos individuais
            for j in range(len(maquinas)):
                
                # Caso exista, adiciona a sua descrição
                if check_names(nome_puro, maquinas[j]) == True:
                    descricao = get_description(maquinas[j])
                    filewriter.writerow([nome_puro,descricao])
                    break
                    
            # Caso não exista, adiciona informação de não exportado
            else:
                descricao = "nao exportado"
                filewriter.writerow([nome_puro,descricao])


In [4]:
def adiciona_novas_colunas():
    df = pd.read_csv('lista.csv')
    
    # remove a coluna de Descrição
    df = df.drop(['Descricao'], axis=1) 
    
    # Adiciona as seguintes colunas
    df['Modelo'] = ''
    df['Setor'] = ''
    df['Sala'] = ''
    
    df.to_csv('nova_lista.csv')

In [27]:
# def adiciona_informacoes(caminho):
caminho = 'infos/Relatório de Microcomputadores agupados por UL (Endereço).csv'
count =0

with open('teste.csv', 'w') as csvfile:
    filewriter = csv.writer(
            csvfile, 
            lineterminator='\n',
            delimiter=',',
            quotechar='|', 
            quoting=csv.QUOTE_MINIMAL
        )
        
        # Cria colunas
        filewriter.writerow(['Patrimonio', 'Patrimonio Antigo', 'Setor'])


with open(caminho, 'r', encoding = "ISO-8859-1") as maquinas:
    df = pd.read_csv('teste.csv')
    for row in maquinas:
#         print(''.join(row))
        if 'U.A.:' in row:
#             print(row)
            setor = row[16:-16]
            print(setor)

FileNotFoundError: File b'teste.csv' does not exist